In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="llama-3.1-8b-instant")

In [2]:
from typing import Annotated, List
import operator
from typing_extensions import Literal
from pydantic import BaseModel,Field
from langchain_core.messages import HumanMessage,SystemMessage
from typing_extensions import TypedDict

In [34]:
class State(TypedDict):
    topic: str
    generated_content: str
    feedback:str
    verdict:str

class EvaluatorState(BaseModel):
    generated_content:str = Field(description='This is the conytent generated on a topic')
    verdict: Literal['good','bad'] = Field(description='This is the verdict of the evaluator')
    feedback:str = Field(description='This tells the feedback of what can be improved')

In [35]:
evaluator = llm.with_structured_output(EvaluatorState)

In [36]:
def generator(state:State):
    if state.get('feedback'):
        content = llm.invoke(
             f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}"
        
        )
    else:
        content = llm.invoke(
            f"Write a joke about {state['topic']}"
        )
    return {"generated_content":content.content}

In [70]:
def evaluator_agent(state:State):
    verdict_state = evaluator.invoke(f"Grade the content {state['generated_content']} as a strict evaluator")
    return {"verdict": verdict_state.verdict.strip(), "feedback": verdict_state.feedback.strip()}

In [71]:
def conditional_route(state:State):
    print("Conditional Called")
    if state['verdict'] == 'good':
        print("Accepted")

        return "Accepted"
    
    print("Rejected")
    return "Rejected + Feedback"

In [72]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

graph_builder = StateGraph(State)

graph_builder.add_node("generator",generator)
graph_builder.add_node("evaluator_agent",evaluator_agent)

graph_builder.add_edge(START,"generator")
graph_builder.add_edge("generator","evaluator_agent")
graph_builder.add_conditional_edges("evaluator_agent",conditional_route, {"Accepted":END,"Rejected + Feedback":"generator"})

graph = graph_builder.compile()


In [50]:
display(Image(graph.get_graph().draw_mermaid_png()))


KeyboardInterrupt: 

In [79]:
state = graph.invoke({"topic": "Tell me something funny"})
print(state["generated_content"])

Conditional Called
Rejected
Conditional Called
Accepted
To improve the quality and clarity of the joke, I'll create a new one that addresses the issues you mentioned. Here's an attempt at a well-researched, structured joke with a stronger punchline:

**Joke:**

Why did Albert Einstein's cat join a band?

**Setup:** You might think that Albert Einstein, the famous physicist, had nothing to do with music. However, his love for simplicity and harmony is actually a great fit for the world of music. Einstein's cat, on the other hand, had a more unconventional passion.

**Punchline:** The cat joined a band because it was a purr-cussionist – it loved to hit the right notes and make some noise, just like Einstein's theory of relativity made waves in the scientific community.

**Improvements:**

1. **Clear context**: The setup establishes the connection between Einstein and music, making the joke more relatable and engaging.
2. **Well-researched topic**: The joke incorporates a bit of trivia ab